# Worksheet 08

Name: Uday Garg

UID: U11302797

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [13]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[6.304325123174699, 5.1748057455004695, 5.895349720514842, 4.817339497721734, 4.857036693940893, 6.1351032454803285, 5.69804821147687, 6.781268177291016, 5.272790328950243, 5.845116572311545]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [14]:
s2 = np.random.normal( 8 , 1 , 10 ).tolist()
print(s2)

[7.941863609114154, 8.962034050897925, 6.922350260759055, 8.016720834595493, 8.458148727327483, 8.791639250174468, 7.462033117893832, 6.557229276486337, 7.535742148825161, 7.663923713040404]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [15]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[7.663923713040404, 7.535742148825161, 6.557229276486337, 5.845116572311545, 5.272790328950243, 7.462033117893832, 8.791639250174468, 6.781268177291016, 5.69804821147687, 6.1351032454803285]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

the data is randomly picked from both different datasets. 

the probability that it comes from one dataset is 1/2.

the mean and var of normal distribution gives us a huge insight.



d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [16]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[6.557229276486337, 5.845116572311545, 5.272790328950243, 6.781268177291016, 5.69804821147687, 6.1351032454803285]
[7.663923713040404, 7.535742148825161, 7.462033117893832, 8.791639250174468]
P(S_1) = 0.6,  P(S_2) = 0.4
mean_1 = 6.04825930199939,  mean_2 = 7.863334557483466
var_1 = 0.2615268702361267,  var_2 = 0.29246847974522316


/Users/uday/Library/Python/3.9/lib/python/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [17]:
from scipy.stats import norm

prob_s0_x = []  # P(S_0 | X_i)
prob_s1_x = []  # P(S_1 | X_i)
prob_x = []    # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x_i = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_0 | X_i) = P(X_i | S_0)P(S_0) / P(X_i)
    prob_s0_x_i = (pdf_i[0] * prob_s[0]) / prob_x_i
    prob_s0_x.append(prob_s0_x_i)

    # P(S_1 | X_i) = P(X_i | S_1)P(S_1) / P(X_i)
    prob_s1_x_i = (pdf_i[1] * prob_s[1]) / prob_x_i
    prob_s1_x.append(prob_s1_x_i)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_0 = " + str(p[1]))
    print("Probability of coming from S_1 = " + str(p[2]))
    print()


point =  7.663923713040404
probability of observing that point if it came from cluster 0 =  7.868374071778253e-09
probability of observing that point if it came from cluster 1 =  1.0811449558401012
point =  7.535742148825161
probability of observing that point if it came from cluster 0 =  1.4412150685438898e-07
probability of observing that point if it came from cluster 1 =  0.7284427243010232
point =  6.557229276486337
probability of observing that point if it came from cluster 0 =  0.2295886270296326
probability of observing that point if it came from cluster 1 =  6.370687884808486e-05
point =  5.845116572311545
probability of observing that point if it came from cluster 0 =  1.12817838604657
probability of observing that point if it came from cluster 1 =  6.230894430148176e-11
point =  5.272790328950243
probability of observing that point if it came from cluster 0 =  0.018801723666261
probability of observing that point if it came from cluster 1 =  1.2543429558737524e-17
point =  7.

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [19]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [sum([x[0] * ((x[1] - mean[0])**2) for x in zip(prob_s0_x, data)]) / sum(prob_s0_x),
       sum([x[0] * ((x[1] - mean[1])**2) for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]


print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.6,  P(S_2) = 0.4
mean_1 = 6.044404679629524,  mean_2 = 7.85488382302688
var_1 = 0.2600682881658137,  var_2 = 0.29927365923999505


g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [20]:
prob_s0_x = []  # P(S_0 | X_i)
prob_s1_x = []  # P(S_1 | X_i)

for p in data:
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))

    # P(S_0 | X_i) = P(X_i | S_0)P(S_0) / P(X_i)
    prob_s0_x_i = (pdf_i[0] * prob_c[0]) / (pdf_i[0] * prob_c[0] + pdf_i[1] * prob_c[1])
    prob_s0_x.append(prob_s0_x_i)

    # P(S_1 | X_i) = P(X_i | S_1)P(S_1) / P(X_i)
    prob_s1_x_i = (pdf_i[1] * prob_c[1]) / (pdf_i[0] * prob_c[0] + pdf_i[1] * prob_c[1])
    prob_s1_x.append(prob_s1_x_i)

# Print updated posterior probabilities
probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_0 = " + str(p[1]))
    print("Probability of coming from S_1 = " + str(p[2]))
    print()


7.663923713040404
Probability of coming from S_0 = 7.925618897354606e-09
Probability of coming from S_1 = 0.9999999920743811

7.535742148825161
Probability of coming from S_0 = 2.1766076845836543e-07
Probability of coming from S_1 = 0.9999997823392315

6.557229276486337
Probability of coming from S_0 = 0.9996609878241548
Probability of coming from S_1 = 0.00033901217584516663

5.845116572311545
Probability of coming from S_0 = 0.9999999998731544
Probability of coming from S_1 = 1.2684559466659643e-10

5.272790328950243
Probability of coming from S_0 = 0.9999999999999968
Probability of coming from S_1 = 3.2784911246832514e-15

7.462033117893832
Probability of coming from S_0 = 1.4236834383010864e-06
Probability of coming from S_1 = 0.9999985763165617

8.791639250174468
Probability of coming from S_0 = 1.3424840792986356e-22
Probability of coming from S_1 = 1.0

6.781268177291016
Probability of coming from S_0 = 0.95043354520376
Probability of coming from S_1 = 0.04956645479623997

5.698

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [21]:
hard_assignments = []  # List to store hard cluster assignments

for p_s0, p_s1 in zip(prob_s0_x, prob_s1_x):
    if p_s0 > p_s1:
        hard_assignments.append(0)  # Assign to cluster 0
    else:
        hard_assignments.append(1)  # Assign to cluster 1

# Print the hard assignments
for i, assignment in enumerate(hard_assignments):
    print(f"Data point {i} belongs to Cluster {assignment}")


Data point 0 belongs to Cluster 1
Data point 1 belongs to Cluster 1
Data point 2 belongs to Cluster 0
Data point 3 belongs to Cluster 0
Data point 4 belongs to Cluster 0
Data point 5 belongs to Cluster 1
Data point 6 belongs to Cluster 1
Data point 7 belongs to Cluster 0
Data point 8 belongs to Cluster 0
Data point 9 belongs to Cluster 0
